In [175]:
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
# fix random seed for reproducibility
np.random.seed(7)

In [2]:
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()
num_class = 10

In [82]:
x_train = ((x_train_origin/255)>0.5)*1
x_test = ((x_test_origin/255)>0.5)*1

In [153]:
def isSafe(X, i, j, visited): 
        # row number is in range, column number 
        # is in range and value is 0  
        # and not yet visited
        row = X.shape[0]
        col = X.shape[1]
        return (i >= 0 and i < row and 
                j >= 0 and j < col and 
                not visited[i][j] and X[i][j] == 0) 

In [167]:
def DFS(X, i, j, visited): 
  
        # These arrays are used to get row and  
        # column numbers of 8 neighbours  
        # of a given cell 
        rowNbr = [-1, -1, -1,  0, 0,  1, 1, 1]; 
        colNbr = [-1,  0,  1, -1, 1, -1, 0, 1]; 
          
        # Mark this cell as visited 
        visited[i][j] = True
  
        # Recur for all connected neighbours 
        for k in range(8): 
            if isSafe(X, i + rowNbr[k], j + colNbr[k], visited): 
                DFS(X, i + rowNbr[k], j + colNbr[k], visited)

In [165]:
def countIslands(X): 
    # Make a bool array to mark visited cells. 
    # Initially all cells are unvisited
    row = X.shape[0]
    col = X.shape[1]    
    visited = [[False for j in range(col)]for i in range(row)] 

    # Initialize count as 0 and travese  
    # through the all cells of 
    # given matrix 
    count = 0
    for i in range(row): 
        for j in range(col): 
            # If a cell with value 1 is not visited yet,  
            # then new island found 
            if visited[i][j] == False and X[i][j] == 0: 
                # Visit all cells in this island  
                # and increment island count 
                DFS(X, i, j, visited) 
                count += 1

    return count

In [170]:
# training dataset with extra feature 
white_area = np.zeros((x_train.shape[0],1))

for i in range (x_train.shape[0]):
    x = x_train[i]
    white_area[i, 0] = countIslands(x)

    
x_train_dfs = x_train.reshape(x_train.shape[0], -1)/255
x_train_dfs = np.append(x_train_dfs, white_area/3, axis = 1)

In [173]:
x_train_dfs.shape

(60000, 785)

In [174]:
y_train = to_categorical(y_train_origin, num_classes=10) 

In [178]:
# create model
model = Sequential()
model.add(Dense(10, input_dim=785, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train_dfs, y_train, epochs=20, batch_size=10)

Epoch 1/20
60000/60000 [==============================] - 10s 159us/step - loss: 1.9792 - acc: 0.6635
Epoch 2/20
60000/60000 [==============================] - 9s 154us/step - loss: 1.4874 - acc: 0.7922
Epoch 3/20
60000/60000 [==============================] - 10s 163us/step - loss: 1.1670 - acc: 0.8205
Epoch 4/20
60000/60000 [==============================] - 9s 152us/step - loss: 0.9598 - acc: 0.8370
Epoch 5/20
60000/60000 [==============================] - 9s 149us/step - loss: 0.8218 - acc: 0.8495
Epoch 6/20
60000/60000 [==============================] - 10s 161us/step - loss: 0.7264 - acc: 0.8587
Epoch 7/20
60000/60000 [==============================] - 10s 161us/step - loss: 0.6575 - acc: 0.8656
Epoch 8/20
60000/60000 [==============================] - 10s 164us/step - loss: 0.6061 - acc: 0.8708
Epoch 9/20
60000/60000 [==============================] - 10s 164us/step - loss: 0.5662 - acc: 0.8757
Epoch 10/20
60000/60000 [==============================] - 10s 160us/step - loss: 0.5